In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [16]:
class PredatorPreyEnvironment:
    def __init__(self, width=100, height=100, boundary_type='none'):
        self.width = width
        self.height = height
        self.boundary_type = boundary_type
        self.agents = []  # (predators, prey) 

    def is_within_boundaries(self, position):
        """
        Check if a given position is within the environment boundaries.
        """
        x, y = position
        return 0 <= x < self.width and 0 <= y < self.height

    def apply_boundary_conditions(self, position):
        """
        Handle agent boundary interactions based on the environment settings.
        """
        x, y = position
        if self.boundary_type == 'reflective':
            x = max(0, min(self.width - 1, x))
            y = max(0, min(self.height - 1, y))
        elif self.boundary_type == 'toroidal':
            x = x % self.width
            y = y % self.height
        # No boundary constraints if boundary_type is 'none'
        return np.array([x, y], dtype=float)

    def add_agent(self, agent):
        """
        Add an agent (prey or predator) to the environment.
        """
        self.agents.append(agent)

    def update(self):
        """
        Update the positions of all agents in the environment.
        Ensure agents do not overlap by adjusting their positions slightly if they get too close.
        """
        for agent in self.agents:
            agent.move()
            agent.position = self.apply_boundary_conditions(agent.position)

        # prevent overlapping
        for i, agent in enumerate(self.agents):
            for j, other_agent in enumerate(self.agents):
                if i != j:
                    distance = np.linalg.norm(agent.position - other_agent.position)
                    min_distance = 6.0 if agent.type == 'predator' or other_agent.type == 'predator' else 2.0  # space glede na velikost entitete
                    if distance < min_distance:  # Min allowed distance between agents
                        direction = agent.position - other_agent.position
                        if np.linalg.norm(direction) == 0:
                            direction = np.random.uniform(-1, 1, 2)  # Random direction if they are exactly on top
                        direction = direction / np.linalg.norm(direction)
                        agent.position += direction * (min_distance - distance) / 2  # Move agents slightly apart

    def animate(self, steps=100):
        """
        Animate the simulation.
        """
        fig, ax = plt.subplots(figsize=(5, 5))
        ax.set_xlim(0, self.width)
        ax.set_ylim(0, self.height)
        ax.set_xlabel('Width')
        ax.set_ylabel('Height')
        ax.set_title('Predator-Prey Environment')

        patches = []
        for agent in self.agents:
            if agent.type == 'prey':
                triangle = plt.Polygon(self.get_agent_shape(agent), closed=True, color='b')
            elif agent.type == 'predator':
                triangle = plt.Polygon(self.get_agent_shape(agent), closed=True, color='orange', alpha=0.8)
            patches.append(triangle)
            ax.add_patch(triangle)

        def update(frame):
            self.update()
            for patch, agent in zip(patches, self.agents):
                patch.set_xy(self.get_agent_shape(agent))
            return patches

        ani = animation.FuncAnimation(fig, update, frames=steps, blit=False, interval=50, repeat=False)
        plt.close(fig)
        return HTML(ani.to_jshtml())

    def get_agent_shape(self, agent):
        """
        Return the vertices of a triangle representing the agent's orientation.
        """
        direction = np.array([1.0, 0.0])  # Default direction
        if hasattr(agent, 'direction'):
            direction = agent.direction
        direction = direction / np.linalg.norm(direction)
        perp_direction = np.array([-direction[1], direction[0]])
        size = 6 if agent.type == 'predator' else 2  # size
        front = agent.position + direction * size  # triangle length
        left = agent.position - direction * 1 + perp_direction * 1  # Triangle base left
        right = agent.position - direction * 1 - perp_direction * 1  # Triangle base right
        return [front, left, right]

In [17]:
class Prey:
    def __init__(self, position, speed=1.5, perception_radius=10.0, separation_weight=1.5, alignment_weight=1.0, cohesion_weight=1.0):
        self.position = np.array(position, dtype=float)
        self.speed = speed
        self.type = 'prey'
        self.direction = np.random.uniform(-1, 1, 2)  # Random initial direction
        self.perception_radius = perception_radius
        self.separation_weight = separation_weight
        self.alignment_weight = alignment_weight
        self.cohesion_weight = cohesion_weight

    def move(self):
        """
        Move the prey based on its neighbors (separation, alignment, cohesion).
        """
        separation = np.zeros(2)
        alignment = np.zeros(2)
        cohesion = np.zeros(2)
        neighbor_count = 0

        for agent in env.agents:
            if agent is not self and agent.type == 'prey':
                distance = np.linalg.norm(agent.position - self.position)
                if distance < self.perception_radius:
                    neighbor_count += 1
                    # Separation: steer away from neighbors
                    separation -= (agent.position - self.position) / distance
                    # Alignment: align with average direction of neighbors
                    alignment += agent.direction
                    # Cohesion: move towards the average position of neighbors
                    cohesion += agent.position

        if neighbor_count > 0:
            separation = separation / neighbor_count * self.separation_weight
            alignment = (alignment / neighbor_count - self.direction) * self.alignment_weight
            cohesion = (cohesion / neighbor_count - self.position) * self.cohesion_weight

        # Avoid predators
        for agent in env.agents:
            if agent.type == 'predator':
                distance = np.linalg.norm(agent.position - self.position)
                if distance < self.perception_radius:
                    separation -= (agent.position - self.position) / distance * 2.0  # Stronger repulsion from predators

        # Combine all the influences
        self.direction += separation + alignment + cohesion
        self.direction = self.direction / np.linalg.norm(self.direction)  # Normalize direction
        self.position += self.direction * self.speed

In [18]:
class Predator:
    def __init__(self, position, speed=2.0, perception_radius=15.0):  # Predators move slightly faster
        self.position = np.array(position, dtype=float)
        self.speed = speed
        self.type = 'predator'
        self.direction = np.random.uniform(-1, 1, 2)  # Random initial direction
        self.perception_radius = perception_radius

    def move(self):
        """
        Move the predator towards the nearest prey.
        """
        nearest_prey = None
        min_distance = float('inf')
        for agent in env.agents:
            if agent.type == 'prey':
                distance = np.linalg.norm(agent.position - self.position)
                if distance < self.perception_radius and distance < min_distance:
                    nearest_prey = agent
                    min_distance = distance

        if nearest_prey is not None:
            # Steer towards the nearest prey
            self.direction = (nearest_prey.position - self.position) / np.linalg.norm(nearest_prey.position - self.position)

        # Add some random movement
        direction_change = np.random.uniform(-0.05, 0.05, 2)  # Small random change to direction
        self.direction += direction_change
        self.direction = self.direction / np.linalg.norm(self.direction)  # Normalize direction
        self.position += self.direction * self.speed

In [20]:
# Create environment instance
env = PredatorPreyEnvironment(width=100, height=100, boundary_type='none')

# Add prey agents to the environment
num_prey = 10
for _ in range(num_prey):
    initial_position = np.random.uniform(0, 100, 2)
    prey = Prey(position=initial_position)
    env.add_agent(prey)

# Add predator agents to the environment
num_predators = 3
for _ in range(num_predators):
    initial_position = np.random.uniform(0, 100, 2)
    predator = Predator(position=initial_position)
    env.add_agent(predator)

# Animate the environment
animation_html = env.animate(steps=100)
animation_html